<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/fine_tune_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

在这个 Colab 中，我们提供了一个使用 Hugging Face 🤗 Transformers 在任意多语言自动语音识别（ASR）数据集上微调 Whisper 的分步指南。这是配套[博客文章](https://huggingface.co/blog/fine-tune-whisper)的更“实践性”版本。如需更深入了解 Whisper、Common Voice 数据集以及微调的理论，建议读者参考该博客文章。

## Introduction

Whisper 是一个用于自动语音识别（ASR）的预训练模型，由 OpenAI 的 Alec Radford 等人于 [2022 年 9 月](https://openai.com/blog/whisper/) 发布。与许多前代模型（如 [Wav2Vec 2.0](https://arxiv.org/abs/2006.11477) 使用未标记音频数据进行预训练）不同，Whisper 在大量**带标签**的音频-转录数据上进行预训练，具体为 68 万小时。这比用于训练 Wav2Vec 2.0 的未标记音频数据（6 万小时）多出一个数量级。此外，其中 11.7 万小时的预训练数据是多语言 ASR 数据。这使得 Whisper 的检查点可应用于超过 96 种语言，其中许多被认为是_低资源_语言。

当预训练数据扩展到 68 万小时的带标签数据时，Whisper 模型展现出对多种数据集和领域的强大泛化能力。预训练检查点在 LibriSpeech ASR 的 test-clean 子集上取得了接近 3% 的词错误率（WER），并在 TED-LIUM 上以 4.7% 的 WER 达到了新的最先进水平（参见 [Whisper 论文](https://cdn.openai.com/papers/whisper.pdf) 的表 8）。Whisper 在预训练期间获得的广泛多语言 ASR 知识可以被用于其他低资源语言；通过微调，预训练检查点可以适配特定数据集和语言，进一步提升这些结果。在这个 Colab 中，我们将展示如何为低资源语言微调 Whisper。

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>图 1：</b> Whisper 模型。其架构遵循标准的基于 Transformer 的编码器-解码器模型。输入到编码器的是对数 Mel 频谱图。编码器的最后隐藏状态通过跨注意力机制输入到解码器。解码器以自回归方式预测文本标记，联合依赖于编码器的隐藏状态和之前预测的标记。图片来源：
<a href="https://openai.com/blog/whisper/">OpenAI Whisper 博客</a>。</figcaption>
</figure>

Whisper 检查点有五种不同模型大小的配置。最小的四种模型在英语或多语言数据上进行训练。最大的检查点仅支持多语言。所有 11 个预训练检查点都可在 [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper) 上获取。以下表格总结了这些检查点，并提供了指向 Hub 上模型的链接：

| 大小     | 层数 | 宽度 | 注意力头数 | 参数量 | 仅英语模型                                           | 多语言模型                                          |
|----------|------|------|------------|--------|------------------------------------------------------|-----------------------------------------------------|
| tiny     | 4    | 384  | 6          | 3900万 | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)    |
| base     | 6    | 512  | 8          | 7400万 | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)     |
| small    | 12   | 768  | 12         | 2.44亿 | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)    |
| medium   | 24   | 1024 | 16         | 7.69亿 | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium)   |
| large    | 32   | 1280 | 20         | 15.5亿 | x                                                    | [✓](https://huggingface.co/openai/whisper-large)    |
| large-v2 | 32   | 1280 | 20         | 15.5亿 | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v2) |
| large-v3 | 32   | 1280 | 20         | 15.5亿 | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v3) |

为了演示，我们将微调多语言版本的 [`"small"`](https://huggingface.co/openai/whisper-small) 检查点，参数量为 2.44 亿（约 1GB）。至于数据，我们将在 [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0) 数据集中的一种低资源语言上进行训练和评估。我们将展示，仅用 8 小时的微调数据，就能在该语言上实现强大的性能。

------------------------------------------------------------------------

\\({}^1\\) Whisper 的名称源自首字母缩写“WSPSR”，代表“Web-scale Supervised Pre-training for Speech Recognition”（网络规模监督预训练语音识别）。

## Prepare Environment

首先，我们来为 Colab 争取一块不错的 GPU！遗憾的是，使用 Google Colab 的免费版本越来越难获得高性能 GPU。不过，使用 Google Colab Pro 通常可以轻松分配到 V100 或 P100 GPU。

要获取 GPU，点击 _运行时_ -> _更改运行时类型_，然后将 _硬件加速器_ 从 _CPU_ 更改为可用的 GPU，例如 _T4_（如果有更好的 GPU 可用，也可选择）。接下来，点击屏幕右上角的 `连接 T4`（或 `连接 {V100, A100}`，如果选择了其他 GPU）。

我们可以验证是否已分配到 GPU 并查看其规格：

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May  8 10:31:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   44C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

我们将使用几个流行的 Python 包来微调 Whisper 模型。我们将使用 `datasets[audio]` 下载和准备训练数据，同时使用 `transformers` 和 `accelerate` 加载和训练 Whisper 模型。我们还需要 `soundfile` 包来预处理音频文件，`evaluate` 和 `jiwer` 来评估模型性能，以及 `tensorboard` 来记录指标。最后，我们将使用 `gradio` 构建一个炫酷的微调模型演示。

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 166.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 184.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 185.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 193.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 173.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

我们强烈建议在训练过程中将模型检查点直接上传到 [Hugging Face Hub](https://huggingface.co/)。Hub 提供以下功能：
- 集成的版本控制：确保训练过程中不会丢失任何模型检查点。
- Tensorboard 日志：跟踪训练过程中的重要指标。
- 模型卡：记录模型的功能及其预期用例。
- 社区：与社区分享和协作的便捷方式！

将笔记本链接到 Hub 非常简单——只需在提示时输入您的 Hub 认证令牌即可。在此找到您的 Hub 认证令牌：[https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)。

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

使用 🤗 Datasets 下载和准备数据非常简单。我们只需一行代码即可下载并准备 Common Voice 的各个数据分割。

首先，请确保您已在 Hugging Face Hub 上接受使用条款：[mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0)。接受条款后，您将获得对数据集的完整访问权限，并能够本地下载数据。

由于印地语是低资源语言，我们将合并 `train` 和 `validation` 分割，以获得大约 8 小时的训练数据。我们将使用 4 小时的 `test` 数据作为隔离测试集：

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
# https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0/viewer/hi
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation", trust_remote_code=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test", trust_remote_code=True)

print(common_voice)

The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


n_shards.json:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


hi_train_0.tar:   0%|          | 0.00/114M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3328it [00:00, 144291.79it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 680it [00:00, 118917.89it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 6540
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2894
    })
})


大多数 ASR 数据集仅提供输入音频样本（`audio`）和对应的转录文本（`sentence`）。Common Voice 包含额外的元数据信息，如 `accent` 和 `locale`，但这些对于 ASR 可以忽略。为了使笔记本尽可能通用，我们在微调时仅考虑输入音频和转录文本，丢弃额外的元数据信息：

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)


DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 6540
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2894
    })
})


## Prepare Feature Extractor, Tokenizer and Data

ASR 流程可以分解为三个阶段：

1. 特征提取器：预处理原始音频输入。
2. 模型：执行序列到序列的映射。
3. 分词器：将模型输出后处理为文本格式。

在 🤗 Transformers 中，Whisper 模型配备了相关的特征提取器和分词器，分别称为 [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor) 和 [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)。

我们将逐一详细介绍如何设置特征提取器和分词器！

### Load WhisperFeatureExtractor

Whisper 特征提取器执行两个操作：
1. 将音频输入填充/截断至 30 秒：短于 30 秒的音频输入将被填充至 30 秒（用静音，即零值），长于 30 秒的音频将被截断至 30 秒。
2. 将音频输入转换为 _log-Mel 频谱图_ 输入特征，这是一种音频的可视化表示，也是 Whisper 模型期望的输入形式。

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>图 2：</b> 从采样音频数组到 log-Mel 频谱图的转换。左图：采样的 1 维音频信号。右图：对应的 log-Mel 频谱图。图片来源：
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment 博客</a>。
</figcaption>

我们将使用默认值从预训练的检查点加载特征提取器：

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
print(feature_extractor)

WhisperFeatureExtractor {
  "chunk_length": 30,
  "dither": 0.0,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}



### Load WhisperTokenizer

Whisper 模型输出一个 _tokenid_ 序列。分词器将这些tokenid映射到对应的文本字符串。对于印地语，我们可以加载预训练的分词器并直接用于微调，无需任何进一步修改。我们只需指定目标语言和任务。这些参数会通知分词器在编码标签序列的开头添加语言和任务标记：

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
print(tokenizer)

### Combine To Create A WhisperProcessor

为了简化特征提取器和分词器的使用，我们可以将两者_封装_到一个 `WhisperProcessor` 类中。这个处理器对象继承了 `WhisperFeatureExtractor` 和 `WhisperTokenizer` 的功能，可以根据需要对音频输入和模型预测进行处理。这样，在训练过程中我们只需跟踪两个对象：`processor` 和 `model`：

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

In [ ]:
print(processor)

### Prepare Data

好的，让我们来打印 Common Voice 数据集的第一个示例，看看数据的格式是什么样的：

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1bfc12b9ee30f73bf143fa237d4ba38488008883c25816876e1a35295c9575d3/hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 5.81611368e-26, -1.48634016e-25, -9.37040538e-26, ...,
        1.06425901e-07,  4.46416450e-08,  2.61450239e-09]), 'sampling_rate': 48000}, 'sentence': 'हमने उसका जन्मदिन मनाया।'}


由于我们的输入音频采样率是 48kHz，在将其传递给 Whisper 特征提取器之前，我们需要将其**下采样**到 16kHz，因为 Whisper 模型期望的采样率是 16kHz。

我们将使用数据集的 [`cast_column`](https://www.google.com/search?q=%5Bhttps://huggingface.co/docs/datasets/package_reference/main_classes.html%3Fhighlight%3Dcast_column%23datasets.DatasetDict.cast_column%5D\(https://huggingface.co/docs/datasets/package_reference/main_classes.html%3Fhighlight%3Dcast_column%23datasets.DatasetDict.cast_column\)) 方法将音频输入设置为正确的采样率。这个操作不会就地更改音频，而是通知 `datasets` 在第一次加载音频样本时**动态地**对其进行重采样：

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

重新加载 Common Voice 数据集中的第一个音频样本时，它将被重采样到所需的采样率：

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1bfc12b9ee30f73bf143fa237d4ba38488008883c25816876e1a35295c9575d3/hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 3.81639165e-17,  2.42861287e-17, -1.73472348e-17, ...,
       -1.30981789e-07,  2.63096808e-07,  4.77157300e-08]), 'sampling_rate': 16000}, 'sentence': 'हमने उसका जन्मदिन मनाया।'}


现在我们可以编写一个函数来准备模型所需的数据了：

1.  我们通过调用 `batch["audio"]` 来加载并重采样音频数据。正如上面解释的那样，🤗 Datasets 会在需要时动态地执行任何必要的重采样操作。
2.  我们使用特征提取器从我们的一维音频数组中计算出 log-Mel 频谱图输入特征。
3.  我们通过使用分词器将文本转录编码为标签 ID。

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

我们可以使用数据集的 `.map` 方法将这个数据准备函数应用到我们所有的训练样本上。参数 `num_proc` 指定了要使用的 CPU 核心数量。设置 `num_proc` 大于 1 将启用多进程处理。如果使用多进程时 `.map` 方法挂起，请将 `num_proc` 设置为 1 并按顺序处理数据集。

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)

Map (num_proc=2):   0%|          | 0/6540 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2894 [00:00<?, ? examples/s]

In [ ]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 6540
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2894
    })
})


In [ ]:
print(len(common_voice["train"][0]["input_features"]),len(common_voice["train"][0]["input_features"][0]))

80 3000


In [ ]:
print(len(common_voice["train"][0]["labels"]))

30


## Training and Evaluation

既然我们已经准备好了数据，现在就可以深入研究训练流程了。

[🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) 将为我们完成大部分繁重的工作。我们只需要做以下几件事：

* **加载预训练的检查点**：我们需要加载一个预训练的检查点，并针对训练对其进行正确的配置。
* **定义数据整理器 (data collator)**：数据整理器接收我们预处理的数据，并准备好供模型使用的 PyTorch 张量。
* **评估指标**：在评估过程中，我们希望使用**词错误率 (WER)** ([https://huggingface.co/metrics/wer](https://huggingface.co/metrics/wer)) 指标来评估模型。我们需要定义一个 `compute_metrics` 函数来处理这个计算。
* **定义训练配置**：这将由 🤗 Trainer 用来定义训练计划。

一旦我们对模型进行了微调，我们将在测试数据上对其进行评估，以验证我们已正确地训练它来转录印地语语音。

### Load a Pre-Trained Checkpoint

我们将从预训练的 Whisper `small` 检查点开始我们的微调过程。我们需要从 Hugging Face Hub 加载该检查点的权重。同样，通过使用 🤗 Transformers，这非常简单！

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

我们可以禁用推理过程中执行的自动语言检测任务，并强制模型以印地语生成。为此，我们将生成配置中的 [`language`](https://www.google.com/search?q=%5Bhttps://huggingface.co/docs/transformers/en/model_doc/whisper%23transformers.WhisperForConditionalGeneration.generate.language%5D\(https://huggingface.co/docs/transformers/en/model_doc/whisper%23transformers.WhisperForConditionalGeneration.generate.language\)) 和 [`task`](https://www.google.com/search?q=%5Bhttps://huggingface.co/docs/transformers/en/model_doc/whisper%23transformers.WhisperForConditionalGeneration.generate.task%5D\(https://huggingface.co/docs/transformers/en/model_doc/whisper%23transformers.WhisperForConditionalGeneration.generate.task\)) 参数设置为相应的值。我们还将把所有的 [`forced_decoder_ids`](https://www.google.com/search?q=%5Bhttps://huggingface.co/docs/transformers/main_classes/text_generation%23transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids%5D\(https://huggingface.co/docs/transformers/main_classes/text_generation%23transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids\)) 设置为 `None`，因为这是设置语言和任务参数的旧方法：

In [ ]:
model.generation_config.language = "hindi"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

序列到序列语音模型的数据整理器是独特的，因为它独立地处理 `input_features` 和 `labels`：`input_features` 必须由特征提取器处理，而 `labels` 必须由分词器处理。

`input_features` 已经通过特征提取器的作用被填充到 30 秒，并转换为固定维度的 log-Mel 频谱图，所以我们只需要将 `input_features` 转换为批处理的 PyTorch 张量。我们使用特征提取器的 `.pad` 方法并设置 `return_tensors=pt` 来完成这个操作。

另一方面，`labels` 是未填充的。我们首先使用分词器的 `.pad` 方法将序列填充到批处理中的最大长度。然后，填充的 tokens 会被替换为 `-100`，这样在计算损失时就不会考虑这些 tokens。之后，我们从标签序列的开头删除 BOS token，因为我们稍后会在训练过程中添加它。

我们可以利用我们之前定义的 `WhisperProcessor` 来执行特征提取器和分词器的操作：

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
print(model.config)

WhisperConfig {
  "_attn_implementation_autoset": true,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mas

In [ ]:
print(model.config.decoder_start_token_id)

50258


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

我们将使用词错误率 (WER) 指标，这是评估 ASR 系统的实际标准指标。有关更多信息，请参阅 WER 的[文档](https://huggingface.co/metrics/wer)。我们将从 🤗 Evaluate 加载 WER 指标：

In [ ]:
import evaluate

metric = evaluate.load("wer")

然后我们只需要定义一个函数，该函数接收我们的模型预测并返回 WER 指标。这个名为 `compute_metrics` 的函数首先将 `label_ids` 中的 `-100` 替换为 `pad_token_id`（撤销我们在数据整理器中应用的步骤，以便在损失计算中正确地忽略填充的 tokens）。然后，它将预测的和标签的 ID 解码为字符串。最后，它计算预测和参考标签之间的 WER：

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

在最后一步，我们定义所有与训练相关的参数。有关训练参数的更多详细信息，请参阅 Seq2SeqTrainingArguments 的[文档](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)。

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=20,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=1000,
eval_strategy=IntervalStrategy.STEPS,
eval_use_gather_objec

**注意**：如果不想将模型检查点上传到 Hub，请设置 `push_to_hub=False`。

我们可以将训练参数以及我们的模型、数据集、数据整理器和 `compute_metrics` 函数一起传递给 🤗 Trainer：

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-33-69786f5d74d5>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


好的，在开始训练之前，我们将保存一次processor对象。由于processor是不可训练的，因此在整个训练过程中它不会发生变化：

In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
!ls -lh /content/whisper-small-hi/

total 1.9M
-rw-r--r-- 1 root root   34K May  8 10:51 added_tokens.json
-rw-r--r-- 1 root root  483K May  8 10:51 merges.txt
-rw-r--r-- 1 root root   52K May  8 10:51 normalizer.json
-rw-r--r-- 1 root root   356 May  8 10:51 preprocessor_config.json
-rw-r--r-- 1 root root  2.2K May  8 10:51 special_tokens_map.json
-rw-r--r-- 1 root root  277K May  8 10:51 tokenizer_config.json
-rw-r--r-- 1 root root 1013K May  8 10:51 vocab.json


### Training

训练大约需要 5 到 10 个小时，具体时间取决于您的 GPU 或分配给此 Google Colab 的 GPU。如果您直接使用此 Google Colab 来微调 Whisper 模型，请务必确保训练不会因长时间不活动而中断。一个简单的解决办法是将以下 JavaScript 代码粘贴到当前标签页的控制台中（右键单击 -> 检查 -> 控制台选项卡 -> 粘贴代码）：



```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

根据给定的训练配置，GPU 峰值内存使用量约为 15.8GB。取决于分配给 Google Colab 的 GPU，您在启动训练时可能会遇到 CUDA `"out-of-memory"` 错误。在这种情况下，您可以逐步将 `per_device_train_batch_size` 减小 2 的倍数，并使用 [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer%23transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps) 参数来弥补。

要启动训练，只需执行以下操作：

In [36]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Step,Training Loss,Validation Loss,Wer
1000,0.064500,0.309772,34.326589
2000,0.005900,0.389534,33.204944
3000,0.000500,0.451666,32.900195
4000,0.000300,0.471887,32.785914


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=4000, training_loss=0.069927669164259, metrics={'train_runtime': 20684.4134, 'train_samples_per_second': 3.868, 'train_steps_per_second': 0.193, 'total_flos': 2.30868320256e+19, 'train_loss': 0.069927669164259, 'epoch': 12.232415902140673})

我们最好的词错误率 (WER) 是 32.0% - 对于 8 小时的训练数据来说还不错！我们可以通过添加适当的标签和 README 信息，使我们的模型在 Hub 上更易于访问。

您可以根据您的数据集、语言和模型名称相应地更改这些值：

In [37]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi - Sanchit Gandhi",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

现在可以将训练结果上传到 Hub 了。要做到这一点，请执行 push_to_hub 命令并保存我们创建的预处理器对象：

In [38]:
trainer.push_to_hub(**kwargs)

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

events.out.tfevents.1746701487.6e4ae841e4a7.308.0:   0%|          | 0.00/42.1k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/weege007/whisper-small-hi/commit/18d6bc457bba155cf2bb3043b9d00538f90a57e0', commit_message='End of training', commit_description='', oid='18d6bc457bba155cf2bb3043b9d00538f90a57e0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/weege007/whisper-small-hi', endpoint='https://huggingface.co', repo_type='model', repo_id='weege007/whisper-small-hi'), pr_revision=None, pr_num=None)

## Building a Demo

既然我们已经微调了模型，现在就可以构建一个演示来展示其 ASR 功能了！我们将使用 🤗 Transformers 的 `pipeline`，它将处理整个 ASR 流程，从预处理音频输入到解码模型预测。

运行下面的示例将生成一个 Gradio 演示，我们可以在其中通过计算机的麦克风录制语音，并将其输入到我们微调的 Whisper 模型中以转录相应的文本：

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="weege007/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

## Closing Remarks

在这篇博客中，我们逐步介绍了如何使用 🤗 Datasets、Transformers 和 Hugging Face Hub 对 Whisper 进行多语言 ASR 微调。有关 Whisper 模型、Common Voice 数据集以及微调背后的理论的更多详细信息，请参阅随附的[博客文章](https://huggingface.co/blog/fine-tune-whisper)。如果您有兴趣微调其他 Transformers 模型（无论是英语还是多语言 ASR），请务必查看 [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition) 中的示例脚本。